In [11]:
import dash
from dash.dependencies import Output, Event
import dash_core_components as dcc
import dash_html_components as html
import plotly
import random
import plotly.graph_objs as go
from collections import deque
import sqlite3
import pandas as pd

#popular topics: google, olympics, trump, gun, usa

app = dash.Dash(__name__)
app.layout = html.Div(
    [   html.H2('Live Twitter Sentiment'),
        dcc.Graph(id='live-graph', animate=True),
        dcc.Interval(
            id='graph-update',
            interval=1*1000
        ),
    ]
)

@app.callback(Output('live-graph', 'figure'),
              events=[Event('graph-update', 'interval')])
def update_graph_scatter():
    try:
        conn = sqlite3.connect('twitter.db')
        c = conn.cursor()
        df = pd.read_sql("SELECT * FROM sentiment WHERE tweet LIKE '%BTC%' ORDER BY unix DESC LIMIT 1000", conn)
        df.sort_values('unix', inplace=True)
        df['sentiment_smoothed'] = df['sentiment'].rolling(int(len(df)/5)).mean()
        df.dropna(inplace=True)

        X = df.unix.values[-100:]
        Y = df.sentiment_smoothed.values[-100:]

        data = plotly.graph_objs.Scatter(
                x=X,
                y=Y,
                name='Scatter',
                mode= 'lines+markers'
                )

        return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[min(X),max(X)]),
                                                    yaxis=dict(range=[min(Y),max(Y)]),)}

    except Exception as e:
        with open('errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')


if __name__ == '__main__':
    app.run_server(debug=True)

 * Restarting with stat


SystemExit: 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [17]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import sqlite3
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from unidecode import unidecode
import time

analyzer = SentimentIntensityAnalyzer()

conn = sqlite3.connect('twitter.db')
c = conn.cursor()

def create_table():
    try:
        c.execute("CREATE TABLE IF NOT EXISTS sentiment(unix REAL, tweet TEXT, sentiment REAL)")
        c.execute("CREATE INDEX fast_unix ON sentiment(unix)")
        c.execute("CREATE INDEX fast_tweet ON sentiment(tweet)")
        c.execute("CREATE INDEX fast_sentiment ON sentiment(sentiment)")
        conn.commit()
    except Exception as e:
        print(str(e))
create_table()

#consumer key, consumer secret, access token, access secret.
ckey="5kA2fb8mRibaiPhiRJOcpXhY8"
csecret="mEeM0gIN4X1fDseoroptwwzyYYrAuTYijPGvdjkGA7TZgbqPkA"
atoken="94995010-Uam2RqiOVuw4Reorml6iQYgTXFGpfKOO9e9QZUCoO"
asecret="RNeCjKmyIHOfhcb2iLwWBvxNIV3rMV3ijvzPs55o7OH3p"



class listener(StreamListener):

    def on_data(self, data):
        try:
            data = json.loads(data)
            tweet = unidecode(data['text'])
            time_ms = data['timestamp_ms']
            vs = analyzer.polarity_scores(tweet)
            sentiment = vs['compound']
            print(time_ms, tweet, sentiment)
            c.execute("INSERT INTO sentiment (unix, tweet, sentiment) VALUES (?, ?, ?)",
                  (time_ms, tweet, sentiment))
            conn.commit()

        except KeyError as e:
            print(str(e))
        return(True)

    def on_error(self, status):
        print(status)


while True:

    try:
        auth = OAuthHandler(ckey, csecret)
        auth.set_access_token(atoken, asecret)
        twitterStream = Stream(auth, listener())
        twitterStream.filter(track=["a","e","i","o","u"])
    except Exception as e:
        print(str(e))
        time.sleep(5)

index fast_unix already exists
1520041039505 RT @Harry_Styles: I just signed the @AMarch4OurLives petition, and you should too. H

https://t.co/elVhLe0zWb 0.0
1520041039480 Grupo de escola e a pior coisa que tem, o povim chato viu 0.0
1520041039466 KKKKKKK PUTS NAO TENHO MSM SCR ICONE https://t.co/twYQ0ngGAL 0.0
1520041039513 RT @chicobarney: Wagninho da Sequela felizao falando de Gleici no paredao. Alem de feio e burro... #bbb18 -0.296
1520041039519 RT @death_lucifer_k: ohayougozaimasu(^O^)

Jin Ri ha, {Tao noJie Ju } {Chu Ji ri} {Er noRi } {Er kakinoRi } {Min Fa razionoRi } {Ping He noRi } {Nu nozenesutonoRi } {Jin Yu noRi } {Jie Na noRi } {sarusanoRi } {zigusopazurunoRi } {San noRi } {Tao noRi } {San Shi San Guan Yin noRi } ra... 0.34
1520041039524 RT @dolfoda: Eu vou votar pra caralho nesse paredao. Paula sera indicada pela sua maior inimiga. E questao de honra fazer ela voltar plenis... 0.0
1520041039469 RT @ciannnnaa: Lmao I love the explore page https://t.co/dZSm3aIS74 0.8442
15

1520041039655 @19Cat_Fish191 @goofballgeorge @4Mischief2 @_TheRealBecky1 @_Wolfie02 @0spfinsib @1776Stonewall @1ofthegoodguyz... https://t.co/ny84KtYqmA 0.0
1520041039640 @_wintergirl93 @womensmarch HaaHaaa...they think Congress will pass a dream act HaHaaHa 0.25
1520041039637 RT @thabrowndoll: UNDER THE STREET LIGHTS AT 9 BE THERE TO CATCH THIS FADE https://t.co/z2jH5p5MmS 0.0
1520041039668 @chicobarney acho ele um saco, mas convenhamos q o cara e bonito apesar da cara de debil kkkkk 0.0
1520041039699 RT @eljabiker: CONFIRMADO: Aterrizo avion venezolano sin registrar pasajeros, con valijas llenas de dolares y cocaina. Aparentemente se reu... -0.5574
1520041039638 RT @kthjjk97: I'M DYING, SEOKJIN SLIPPING AND ALMOST FALLING BUT HOLDING HIMSELF UP IN TIME WILL NEVER NOT BE FUNNY I LOVE AN ICON #JinDay... 0.918
1520041039687 RT @CharleneCac: I heard a West Virginian teacher with over 15 years of experience say she earned $45,000 a year. 

I know people with less... 0.0
1520041039614 RT @

1520041040737 nao vou falar nada, qnd ver a merda que ta fazendo eu vou fazer questao de mandar ir se foder 0.0
1520041040684 @NotEdelweiss Mom, can you see how handsome I am  0.4939
1520041040712 mais do que uma
os meus pais
Ines https://t.co/uDGt0kcSEM 0.0
'text'
1520041041669 RT @trvisXX: Abel new album is scray.  It's like when I first heard him for the first time. Fuckkkkkk !!!!! 0.5673
'text'
1520041041659 RT @BobPolglaze: Giannis just got a tech for having to tell the refs the rules. Dope 0.0
1520041041489 RT @QueenNisaJ: @ainturae When will we realize stripping is our destiny https://t.co/Er15D6OBAa 0.0
1520041041634 RT @EdKrassen: You have no right calling students "immature" for speaking their mind, if you support a President who calls his opponents na... 0.128
1520041041615 Agree!... https://t.co/Fo70aiAV3e 0.0
1520041041636 RT @Deon_Cockrell: @JanainOz @TuckerCarlson Same everywhere else. Criminals don't give a damn about your goody2shoe gun laws. -0.5916
1520041041652 RT @

1520041042609 Literally thought I did more damage at Victoria secret, but I did pretty well honestly  0.8659
1520041042652 RT @carlosecaicedo: Somos solidarios con @petrogustavo ante la cobarde agresion de la que fue objeto; manifiesto mi rechazo al ataque en Cu... 0.0
1520041042689 RT @AmericanGent69: Date: Do you want to meet at that All You Can Eat buffet for dinner?
Me: I always eat healthy, so I never go to those,... 0.4588
1520041042603 RT @ZoeSexLife: I want  a dick in my ass https://t.co/tGd2iTzg04 -0.7579
1520041042661 RT @valerialynch: URGENTE! Un bebe de 2 anos padece de cancer necesita un medicamento : carboplatino 450 mg  ... + 584244636611.  Mary hern... -0.69
1520041042682 @N2QvM4VSI6230NV yotsuchiyan
kokomadeTong zi?(o'Wang `o)
Ben Dang nine
Jin kunara, Ri "JUMPde
Yi Xu niLe shinderune.  0.0
1520041042722 RT @intel: After losing a friend to cancer, two engineers created a revolutionary way to detect the deadly disease. -0.4215
1520041042648 @barberDAY Nah I use limes -0

1520041043700 RT @MaddowBlog: Scott Lloyd, director of the Office of Refugee Resettlement, tries to prevent teen girls under his control from acessing ab... 0.0258
1520041043609 im a gemini rising so like this is accurate af. im also an aquarius venus rose is my shit. im a pisces sun but neve... https://t.co/vpOBbkxPQw -0.0463
1520041044662 RT @valehomossexual: "eu nao gosto de strogonoff" https://t.co/cHSUT5vzzB 0.0
'text'
1520041044570 RT @PQPJUNATICA: Gleici ta lindissimaaaaaa e nao choca ninguem  #bbb18 https://t.co/4ErwcZBE8X 0.0
1520041044528 RT @BTSNewsBrasil: [TRANS] "Ja e o 5deg MV!" #BTS "Not Today" MV surperou os 200M de views! 

@BTS_twt #BTSARMY #BestFanArmy #iHeartAwards ht... 0.0
1520041044666 RT @DimeAlets: Encontrada sola en medio de la guerra de Siria, aun asi sonrie a la gente que pasa y apenas sabe lo que esta sucediendo, ?Us... 0.0
'text'
1520041044668 RT @lottieIine: I CANT BELIEVE KEITH THREW HANDS WITH IVERSON,, FUCKED UP HIS EYE AND THEN MOVED TO THE DESERT ..G

1520041045612 RT @lxwissss: when you've just woke up and your mum is asking you a million questions  https://t.co/r01VJDBFAl 0.0
1520041045614 RT @prettytaes: "Hello" he says. He's not usually so nice to people because really jungkook didn't give a shit about anyone but the kid loo... 0.5057
1520041045632 RT @LahLahBean: #wednesdaywit From a passenger ship, everyone can see a bearded man on a small island who is shouting and desperately wavin... -0.3818
1520041045700 A professora indicou um filme na aula de hoje, e quando eu chego em casa, advinha? Ta passando o filme na tv 0.0
1520041045690 RT @BigHitEnt: [gisa] "beolsseo daseos beonjjae"... #bangtansonyeondan 'Not Today' MV 2eogbyu dolpa
https://t.co/hE3gvuVUrP 0.0
1520041045693 I'm using Snap + for Snapchat to add new features to Snapchat for FREE! https://t.co/p1d4Byr2TP 0.6514
1520041045590 RT @lavpurple28: "@imbeatriz26: AND IT'S FINALLY A WRAP!  https://t.co/JLQkSgBWFS" 0.0
1520041045657 I just got Free Instagram Followers on  h

1520041047583 ay amor mio q buenos recuerdos https://t.co/LKR7lUynEN 0.6124
1520041047641 RT @bchapman151: I have 7000 followers - can I get some serious RETWEET ACTION ON THIS ONE??

PLEASE??

This kid @ColtonHaab has had his ac... 0.5709
1520041047680 RT @Camarologo: Grato encuentro con el Presidente de la Federacion de Camaras de Comercio en Sinaloa, Carlos Pardini Soberanes. ACORDAMOS d... 0.0
1520041047602 RT @mrmopar281: So these 2 black guys today on the train were talking not even that loud and said "nigga", when this white lady turns aroun... 0.0
'text'
1520041046462 EL ITS. @ITSPELILEO  TE INVITA A FORMAR PARTE DE NUESTRAS CARRERAS DE EXITO OCUPACIONAL:  DISENO DE MODAS, DESARROL... https://t.co/3ingvzMtVn 0.0
1520041047722 RT @CraveMyThoughts: I'm such a mom when I'm boo'd up or in a relationship like "did you eat today? Was it enough? Are you okay, how's your... 0.5803
1520041047592 RT @VideoGamerRob: I seriously can't believe we're back to blaming video games for real-worl

1520041048677 RT @agustinwinner: No hubo chape damona vOY A ROMPER ALGO -0.296
1520041048723 RT @traytrayolay: Cash app is so convenient. I don't even have to speak to my bm anymore just "here mf" thats what I write on the explanati... 0.0
1520041048640 vai fazer formatura ? -- Nao sei, ainda to pensando https://t.co/lwoLYw6aQd 0.0
1520041048666 - Sana has most confidence in her pupils.
- Her specialty is untangling knots.
- Sana has a scar from when she went... https://t.co/KZ1VbZz9vc 0.5563
1520041048564 RT @cocochanbaek: a beautiful date 
#chanbaek #iHeartAwards #BestFanArmy #EXOL @weareoneEXO https://t.co/F8QpXdTGtY 0.5994
1520041048566 So quero presente de 1.000 pra cima... Obg [?] https://t.co/e4uFYZSAdG 0.0
1520041048667 RT @CatrionaLaing1: Good discussions with No 10 officials on prospects for #Zimbabwe. It's been a busy and productive week. Now on my way b... 0.4404
1520041048688 RT @repjohnlewis: I stand with @Delta 0.0
1520041048651 An Ji (A)ROOM Pei Xin Zhong !!
[(SINGING B

1520041050632 why do i even get my hopes up 0.4215
1520041050440 He's a good boy [?] https://t.co/KxGitNYB4q 0.4404
1520041050656 RT @_mindlessness_: "Todo lo que siempre he querido es acabar con mi vida, pero soy una cobarde, asi que me hago dano en lugares que la gen... 0.296
1520041050584 @MindOfRez hey Rez can you add me on PlayStation my psn is terrisor13 I see your videos and I want to know how it f... https://t.co/ox3ZUbAh09 0.0772
1520041050610 RT @MajuCejas_: Yo hablandole a la cama https://t.co/BYaD64N5bG 0.0
1520041050612 huyupuyusanJia  Pei Xin Zhong !!
https://t.co/N6xGa2FS9V 0.0
1520041050640 RT @SincerelyTumblr: Me when I feel myself breaking out https://t.co/JbPkUIThjy 0.0
1520041050623 RT @GugsM: I really don't like that song  I can appreciate the big pop culture moment but wow I never want to hear it. https://t.co/cP... 0.7204
1520041050583 @Blaney is just too fast...why are you cheering for anyone else? Just become a fan already, you know you want to. #NASCARGoesWest

KeyboardInterrupt: 

In [13]:
!pip install vaderSentiment